In [1]:
import pandas as pd
import numpy as np

$Y = median\_house\_value \Rightarrow$ Verkliga värden, målvariabel

$X = designmatris \Rightarrow$ Alla regressorer, shape

$\hat{Y} = \hat{\beta}_0 + \hat{\beta}_1X_1 + \hat{\beta}_2X_2 + \hat{\beta}_3X_3 ... + \hat{\beta}_dX_d \Rightarrow$ Modellens prediktion för varje observation

$e_i = Y_i - \hat{Y}_i \Rightarrow$ Residual för observation $i$

$SSE = \sum_{i=1}^{n} e_i^2 \Rightarrow$ Sum of Squared Errors - totala felet för modellen

$S_{yy} = \sum(Y_i - \bar{Y})^2 = \sum{Y_{i}}^2 - \frac{(\sum{Y_i})^2}{n} \Rightarrow$ Totala variationen i Y

$SSR = S_{yy} - SSE \Rightarrow$ Regression Sum of Squares - variation som modellen förklarar

$R^2 = \frac{SSR}{S_{yy}} \Rightarrow$ Bestämningskoefficient - hur mycket av Y som modellen förklarar

$\hat{\sigma}^2 = \frac{SSE}{n-d-1} \Rightarrow$ Estimering av residualvarians

$RMSE = \sqrt{\frac{SSE}{n-d-1}} \Rightarrow$ Root Mean Squared Error - spridning i samma enhet som Y (använder $\sqrt{\frac{SSE}{n-d-1}}$ istället för $\sqrt{\frac{SSE}{n}}$ då den senare används i större utsträckning inom ML och uppgiften skulle vara en statistisk analys)

$\hat{\beta} = (X^TX)^{-1}X^TY \Rightarrow$ OLS-koefficienter - beräkning av alla $\beta$ från datan

$C = \hat{\sigma}^2(X^TX)^{-1} \Rightarrow$ Varians och kovarians för $\hat{\beta}$, används i t-test och konfidensintervall

$t_i = \frac{\hat{\beta}_i}{\hat{\sigma}\sqrt{c_{ii}}} \Rightarrow$ t-statistik - Testar om en enskild koefficient skiljer sig signifikant från 0

$F = \frac{SSR/d}{\hat{\sigma}^2} \Rightarrow$ Testar om alla regressorer tillsammans är signifikanta

$\hat{\beta}_i \pm t_{\alpha/2, n-d-1}\hat{\sigma}\sqrt{c_{ii}} \Rightarrow$ Intervallet för $\beta$ med viss säkerhetsnivå

$r = \frac{Cov(X_a,X_b)}{\sqrt{Var(X_a)Var(X_b)}} \Rightarrow$ Pearson $r$ - Mäta linjärt beroende mellan två regressorer

$n$ = antal observationer  
$d$ = antal regressorer  
$c_{ii}$ = diagonalelement i varians-kovariansmatrisen $C$  
Alla formler bygger direkt på residualerna $e_i$ och $X$-matrisen

Datasetet läses in som en dataframe och aggregeras med funktionen aggregate från linreg.py till en observation per geografisk punkt (longitude, latitude) då flera koordinater förekommer mer än en gång, som sedan byts ut till distance_to_centroid istället för att latitud och longitud ska mätas individuellt. Additiva variabler summeras medan nivåvariabler ersätts av det nya medelvärdet och på detta vis tappar vi inte någon precision i modellen. Den kategoriska variabeln ocean_proximity var konstant inom varje grupp vilket även verifierades och möjliggjorde oförändrad inkludering efter aggregering där första värdet per grupp valdes, detta valet påverkade dock alltså inget då alla grupper bara hade samma värden så man hade lika gärna kunnat välja vilket som av värdena inom gruppen.

Därefter one-hot-encodas dataframen med funktionen prepare_features från linreg.py där första kategorin droppas, i detta fallet <1h from ocean som var den näst mest förekommande på drygt 1/3 av grupperna, vilket även är det rimligaste valet då tre av de återstående fyra kategorierna omfattas av <1h from ocean även om de har mer specifika beskrivningar.

In [2]:
from linreg import LinearRegression, aggregate, build_X_Y
housing = pd.read_csv('housing.csv')
aggregated = aggregate(housing)

def prepare_features(df, cat_cols=None, drop_first=True):
    """
    One-hot encodar angivna kategoriska kolumner och returnerar 
    dataframe redo för regression tillsammans med lista på feature-namn.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame som ska transformerad
    cat_cols : list of str
        Kolumner som ska one-hot encodas
    drop_first : bool, default True
        Drop first category för att undvika multikollinearitet
    
    Returns
    -------
    df_prepared : pd.DataFrame
        DataFrame med en-hot encodade kolumner
    feat_cols : list of str
        Lista på kolumnnamn som ska användas som X
    """

    df_prepared = df.copy()

    if cat_cols is not None:
        df_prepared = pd.get_dummies(df_prepared, columns=cat_cols, drop_first=drop_first)
    
    feat_cols = [col for col in df_prepared.columns if col not in ['median_house_value']]
    
    return df_prepared, feat_cols

aggregated, X_cols = prepare_features(aggregated, cat_cols=['ocean_proximity'], drop_first=True)
aggregated


,housing_median_age,median_income,median_house_value,total_rooms,total_bedrooms,population,households,distance_to_centroid,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,52.0,3.0147,94600.0,1820.0,300.0,806.0,270.0,6.588639,False,False,False,True
1,19.0,1.9797,85800.0,2672.0,552.0,1298.0,478.0,7.499125,False,False,False,True
2,17.0,3.0313,103600.0,2677.0,531.0,1244.0,456.0,7.530660,False,False,False,True
3,36.0,2.5179,79000.0,2349.0,528.0,1194.0,465.0,6.639629,False,False,False,True
4,52.0,2.3571,111400.0,2217.0,394.0,907.0,369.0,6.553643,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
12585,17.0,1.6509,85700.0,720.0,174.0,333.0,117.0,5.571844,True,False,False,False
12586,19.0,1.2750,56100.0,2570.0,820.0,1431.0,608.0,5.988815,True,False,False,False
12587,17.0,1.6154,87500.0,2809.0,635.0,83.0,45.0,5.532627,True,False,False,False
12588,19.0,1.8200,80100.0,7650.0,1901.0,1129.0,463.0,5.417262,True,False,False,False


Bygger X- och Y-matriser för regression och lägger till intercept $\beta_0$

In [3]:

X, Y = build_X_Y(aggregated, X_cols)
n = X.shape[0]
d = X.shape[1] - 1


kör OLS-formeln samt beräkning av grundläggande statistik så som $\hat{\sigma}^2$ och $RMSE$

In [4]:
model = LinearRegression(X, Y)
print(f'modellens beta-värden: {model.b_est}')         # modellens beta-värden
print(f'modellens prediktioner: {model.Y_est}')        # modellens prediktioner
print(f'modellens residualer: {model.residuals}')      # residualer
print(f'modellens SSE: {model.SSE}')                   # sum of squared errors
print(f'modellens varians: {model.variance}')          # modellens varians
print(f'modellens STD: {model.STD}')                   # modellens standardavvikelse
print(f'modellens RMSE: {model.RMSE}')                 # modellens RMSE (nästan samma som denna modellens STD p.g.a. stort n-tal)

modellens beta-värden: [ 6.19054104e+04  8.30952663e+02  3.94841132e+04 -9.16374132e-01
  2.76050717e+01 -2.27535191e+01  4.49473667e+01 -6.51138526e+03
 -6.78146968e+04  1.86180843e+05 -8.14028859e+02  2.18830766e+04]
modellens prediktioner: [203539.78691352 133653.81266593 172963.62840736 ...  51063.69629045
  87054.90521205  57256.33266719]
modellens residualer: [-108939.78691352  -47853.81266593  -69363.62840736 ...   36436.30370955
   -6954.90521205    9643.66733281]
modellens SSE: 50237620022104.9
modellens varians: 3994086501.9959373
modellens STD: 63198.78560538911
modellens RMSE: 63168.65986175059


In [5]:

print(f'modellens varians-kovariansmatris: {model.C}') # hur osäkra modellens koefficienter är
print(f'modellens t-statistik: {model.t_stats}')       # mäter signifikans för varje koefficient
print(f'modellens p-värden: {model.p_values}')         # sannolikheten att koefficienten egentligen är 0; lågt värde = signifikant

modellens varians-kovariansmatris: [[ 1.05030989e+07 -9.41779748e+04 -7.46372435e+05  2.17472034e+02
  -1.11113703e+03 -4.34974259e+02  3.37271137e+02 -1.01772260e+06
  -2.59995245e+06 -1.22700218e+06  2.19004967e+05 -6.69474575e+05]
 [-9.41779748e+04  2.77309729e+03  2.18864255e+03  4.33997944e+00
   1.25622153e+01  7.11938215e-01 -3.33471604e+01  2.65003758e+03
   1.22780096e+04 -4.03587018e+04 -1.97461157e+04 -2.56201215e+03]
 [-7.46372435e+05  2.18864255e+03  1.35869920e+05 -9.42285321e+01
   3.12176111e+02  3.77317428e+01  3.81072951e+01  2.45007348e+04
   2.14128096e+05  1.91959179e+05 -1.48600732e+04  3.10781367e+04]
 [ 2.17472034e+02  4.33997944e+00 -9.42285321e+01  2.82490443e-01
  -6.75697950e-01 -5.42069020e-02 -4.14192274e-01 -1.97632942e+01
  -1.27129507e+02 -9.62940324e+01 -5.65176148e+01 -1.44470597e+01]
 [-1.11113703e+03  1.25622153e+01  3.12176111e+02 -6.75697950e-01
   1.85158569e+01  4.93689626e-01 -1.78271736e+01 -6.61087121e+01
  -1.81004480e+02 -2.20794698e+03  2.

| Index | Variabel | p-värde | %  |
|:-----:|----------|---------|----:|
| 0 | intercept | 2.51e-79 | $\lt$ 0,01% |
| 1 | longitude | 1.16e-86 | $\lt$ 0,01% |
| 2 | latitude | 2.97e-85 | $\lt$ 0,01% |
| 3 | housing_median_age | 9.48e-54 | $\lt$ 0,01% |
| 4 | median_income | 0.00e+00 | $\ll$ 0,01% |
| 5 | total_rooms | 1.03e-01 | $\approx$ 10,3% |
| 6 | total_bedrooms | 6.72e-16 | $\lt$ 0,01% |
| 7 | population | 1.50e-198 | $\lt$ 0,01% |
| 8 | households | 7.48e-13 | $\lt$ 0,01% |
| 9 | INLAND | 1.35e-115 | $\lt$ 0,01% |
| 10 | ISLAND | 1.36e-09 | $\lt$ 0,01% |
| 11 | NEAR_BAY | 4.12e-04 | $\approx$ 0,04% |
| 12 | NEAR_OCEAN | 9.30e-10 | $\lt$ 0,01% |


Detta är då signifikansen för de olika kolumnerna där vi då kan se att median income visar extrem statistisk signifikans till median house value, total rooms har runt .103 och är alltså inte något vidare signifikant och resterande har hög signifikans där near bay är en utstickare (fortfarande väldigt hög signifikans men märkbart lägre än de andra)

In [6]:
print(f'Medelvärdet av hurpriserna: {model.Y_mean}')
print(f'Modellens Syy: {model.Syy}')
print(f'Modellens SSR: {model.SSR}')

Medelvärdet av hurpriserna: 201092.05267625765
Modellens Syy: 170054148897735.6
Modellens SSR: 119816528875630.69


In [7]:
print(f'Modellens F_stat: {model.F_stat}')
print(f'Modellens p_value: {model.F_p_value}')
print(f'Modellens R2: {model.R2}')

Modellens F_stat: 2727.1346553268777
Modellens p_value: 0.0
Modellens R2: 0.7045786865669711


In [8]:
# Hämta konfidensintervall
intervals = model.confidence_intervals(alpha=0.05)  # 95% CI

# Skriv ut
for i, (low, high) in enumerate(intervals):
    print(f"Koef {i} ({X_cols[i-1] if i>0 else 'intercept'}): {low:.3f} — {high:.3f}")


Koef 0 (intercept): 55552.853 — 68257.968
Koef 1 (housing_median_age): 727.731 — 934.175
Koef 2 (median_income): 38761.590 — 40206.636
Koef 3 (total_rooms): -1.958 — 0.125
Koef 4 (total_bedrooms): 19.171 — 36.040
Koef 5 (population): -24.212 — -21.295
Koef 6 (households): 34.985 — 54.910
Koef 7 (distance_to_centroid): -7584.070 — -5438.701
Koef 8 (ocean_proximity_INLAND): -70669.419 — -64959.974
Koef 9 (ocean_proximity_ISLAND): 130689.798 — 241671.888
Koef 10 (ocean_proximity_NEAR BAY): -5194.342 — 3566.284
Koef 11 (ocean_proximity_NEAR OCEAN): 18194.072 — 25572.081


In [33]:
numeric_cols = [
    'distance_to_centroid',
    'housing_median_age',
    'median_income',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_house_value'
]

idx_to_name = dict(enumerate(numeric_cols))

X_full = aggregated[numeric_cols].to_numpy(dtype=float)

raw_corr = model.pearson_numeric(X_full)

corr_df = pd.DataFrame(
    raw_corr,
    columns=['idx1', 'idx2', 'Pearson_r']
)

corr_df['Var1'] = corr_df['idx1'].map(idx_to_name)
corr_df['Var2'] = corr_df['idx2'].map(idx_to_name)

target_corr = corr_df[corr_df['Var2'] == 'households']
corr_df

,idx1,idx2,Pearson_r,Var1,Var2
0,0,0,1.000000,distance_to_centroid,distance_to_centroid
1,0,1,-0.055704,distance_to_centroid,housing_median_age
2,0,2,-0.048796,distance_to_centroid,median_income
3,0,3,-0.018246,distance_to_centroid,total_rooms
4,0,4,-0.023695,distance_to_centroid,total_bedrooms
5,0,5,-0.069658,distance_to_centroid,population
6,0,6,-0.036332,distance_to_centroid,households
7,0,7,-0.055732,distance_to_centroid,median_house_value
8,1,1,1.000000,housing_median_age,housing_median_age
9,1,2,-0.113743,housing_median_age,median_income


In [15]:
aggregated2 = aggregate(housing)


df_cat, cat_cols = prepare_features(
    aggregated2,
    cat_cols=['ocean_proximity'],
    drop_first=False
)
cat_dummy_cols = [c for c in cat_cols if c.startswith('ocean_proximity_')]


X_cat = df_cat[cat_dummy_cols].to_numpy()
X_num = df_cat[numeric_cols].to_numpy()

raw_cat_corr = model.pearson_categorical_numeric(X_cat, X_num)


corr_cat_df = pd.DataFrame(
    raw_cat_corr,
    columns=['cat_idx', 'num_idx', 'Pearson_r']
)

cat_idx_to_name = dict(enumerate(cat_dummy_cols))
num_idx_to_name = dict(enumerate(numeric_cols))

corr_cat_df['Category'] = corr_cat_df['cat_idx'].map(cat_idx_to_name)
corr_cat_df['Variable'] = corr_cat_df['num_idx'].map(num_idx_to_name)


corr_cat_df.sort_values(
    by=['Pearson_r'],
    key=abs,
    ascending=[False]
)

target_corr3 = corr_cat_df[corr_cat_df['Variable'] == 'median_house_value']
target_corr3.sort_values(
    by=['Pearson_r'],
    key=abs,
    ascending=[False]
)


,cat_idx,num_idx,Pearson_r,Category,Variable
15,1,7,-0.551185,ocean_proximity_INLAND,median_house_value
7,0,7,0.330131,ocean_proximity_<1H OCEAN,median_house_value
39,4,7,0.201383,ocean_proximity_NEAR OCEAN,median_house_value
31,3,7,0.163675,ocean_proximity_NEAR BAY,median_house_value
23,2,7,0.030759,ocean_proximity_ISLAND,median_house_value
